In [2]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array

# 1. Filter only Normal (N) and Diabetic (D)
dataset_dir = r"C:\projects\ip\ML-project-retina\datasets"
selected_classes = {'N': 'normal', 'D': 'diabetes'}

file_paths, labels = [], []
for label, folder in selected_classes.items():
    folder_path = os.path.join(dataset_dir, folder)
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            file_paths.append(os.path.join(folder_path, filename))
            labels.append(label)

label_map = {'N': 0, 'D': 1}
numeric_labels = [label_map[lbl] for lbl in labels]

# 2. Preprocessing function
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None: return None
    green = image[:, :, 1]
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    green_clahe = clahe.apply(green)
    resized = cv2.resize(green_clahe, (224, 224))
    normalized = resized.astype(np.float32) / 255.0
    return np.expand_dims(normalized, axis=-1)  # For CNN input

# 3. Preprocess all images
X, y = [], []
for path, label in zip(file_paths, numeric_labels):
    img = preprocess_image(path)
    if img is not None:
        X.append(img)
        y.append(label)

X = np.array(X)
y = np.array(y)
print(f"Total images: {X.shape[0]}, Shape: {X.shape[1:]}")

# 4. Split into train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 5. Convert labels
y_train_cat = to_categorical(y_train, num_classes=2)
y_test_cat = to_categorical(y_test, num_classes=2)


Total images: 4481, Shape: (224, 224, 1)


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications

# Simple CNN
def simple_cnn():
    model = models.Sequential([
        layers.Input(shape=(224, 224, 1)),
        layers.Conv2D(32, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(2, activation='softmax')
    ])
    return model

# Transfer Learning Wrapper
def transfer_model(base_model_fn):
    base = base_model_fn(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    base.trainable = False
    model = models.Sequential([
        layers.Input(shape=(224, 224, 1)),
        layers.Conv2D(3, (3,3), padding='same'),  # Convert 1 channel to 3
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dense(2, activation='softmax')
    ])
    return model


In [12]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def train_and_evaluate(model_fn, name):
    print(f"\n🧠 Training: {name}")
    model = model_fn()
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(X_train, y_train_cat, epochs=10, batch_size=4, validation_split=0.1,
              callbacks=[EarlyStopping(monitor='val_loss', patience=3)])

    y_pred = model.predict(X_test)
    y_pred_labels = np.argmax(y_pred, axis=1)

    acc = accuracy_score(y_test, y_pred_labels)
    prec = precision_score(y_test, y_pred_labels)
    rec = recall_score(y_test, y_pred_labels)
    f1 = f1_score(y_test, y_pred_labels)

    print(classification_report(y_test, y_pred_labels, target_names=['Normal', 'Diabetic']))
    return {'Model': name, 'Accuracy': acc, 'Precision': prec, 'Recall': rec, 'F1-Score': f1}


In [13]:
results = []
results.append(train_and_evaluate(simple_cnn, "Simple CNN"))
results.append(train_and_evaluate(lambda: transfer_model(applications.VGG16), "VGG16"))
results.append(train_and_evaluate(lambda: transfer_model(applications.ResNet50), "ResNet50"))
results.append(train_and_evaluate(lambda: transfer_model(applications.EfficientNetB0), "EfficientNetB0"))

# 📊 Results Table
import pandas as pd
results_df = pd.DataFrame(results)
print("\n📈 Performance Comparison:")
print(results_df.to_string(index=False))



🧠 Training: Simple CNN


ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[359552,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2]

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.